Imports

In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Loading and Pre-Processing the diabetes dataset 

In [2]:
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Defining Huber Loss Function

In [3]:
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = np.abs(error) <= delta
    small_error_loss = 0.5 * error**2
    large_error_loss = delta * (np.abs(error) - 0.5 * delta)
    return np.where(is_small_error, small_error_loss, large_error_loss)


Defining the Gradient of the Huber Loss function

In [4]:
def huber_loss_gradient(X, y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = np.abs(error) <= delta
    small_error_gradient = -error
    large_error_gradient = -delta * np.sign(error)
    return np.where(is_small_error, small_error_gradient, large_error_gradient) @ X / len(y_true)

Defining the Gradient Descent Algorithm

In [8]:
def gradient_descent(X, y, learning_rate=0.01, epochs=1000, delta=1.0):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0

    for epoch in range(0,1000):
        y_pred = X @ weights + bias
        loss = huber_loss(y, y_pred, delta).mean()
        gradient = huber_loss_gradient(X, y, y_pred, delta)
        
        # Update weights and bias
        weights -= learning_rate * gradient
        bias -= learning_rate * gradient.mean()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')

    return weights, bias


Defining the Prediction Functino

In [10]:
def predict(X, weights, bias):
    return X @ weights + bias


Training the Model

In [11]:
weights, bias = gradient_descent(X_train, y_train)

Epoch 0, Loss: 153.23654390934846
Epoch 100, Loss: 153.21858249435303
Epoch 200, Loss: 153.20062107935763
Epoch 300, Loss: 153.1826596643622
Epoch 400, Loss: 153.1646982493668
Epoch 500, Loss: 153.1467368343714
Epoch 600, Loss: 153.12877541937598
Epoch 700, Loss: 153.1108140043806
Epoch 800, Loss: 153.09285258938516
Epoch 900, Loss: 153.07489117438976


Evaluating the Model

In [12]:
y_pred_train = predict(X_train, weights, bias)
y_pred_test = predict(X_test, weights, bias)
# Mean Absolute Error (MAE)
mae_train = np.mean(np.abs(y_train - y_pred_train))
mae_test = np.mean(np.abs(y_test - y_pred_test))
print(f'Training MAE: {mae_train}')
print(f'Test MAE: {mae_test}')


Training MAE: 153.55692975939434
Test MAE: 145.89527536530764
